# Big Data Platform
## Assignment 2: MapReduce

**By:**  

Eyal Michaeli, 207380528
Tzach Labroni, 302673355

<br><br>

**The goal of this assignment is to:**
- Understand and practice the details of MapReduceEngine

**Instructions:**
- Students will form teams of two people each, and submit a single homework for each team.
- The same score for the homework will be given to each member of your team.
- Your solution is in the form of a Jupyter notebook file (with extension ipynb).
- Images/Graphs/Tables should be submitted inside the notebook.
- The notebook should be runnable and properly documented. 
- Please answer all the questions and include all your code.
- You are expected to submit a clear and pythonic code.
- You can change functions signatures/definitions.

**Submission:**
- Submission of the homework will be done via Moodle by uploading a Jupyter notebook.
- The homework needs to be entirely in English.
- The deadline for submission is on Moodle.
- Late submission won't be allowed.
  
  
- In case of identical code submissions - both groups will get a Zero. 
- Some groups might be selected randomly to present their code.

**Requirements:**  
- Python 3.6 should be used.  
- You should implement the algorithms by yourself using only basic Python libraries (such as numpy,pandas,etc.)

<br><br><br><br>

**Grading:**
- Q1 - 5 points - Initial Steps
- Q2 - 50 points - MapReduceEngine
- Q3 - 30 points - Implement the MapReduce Inverted index of the JSON documents
- Q4 - 5 points - Testing Your MapReduce
- Q5 - 10 points - Final Thoughts 

`Total: 100`

**Prerequisites**

In [476]:
# example
#!pip install --quiet zipfile36

**Imports**

In [477]:
# general
import os
import random
import warnings
import concurrent
import sqlite3
import traceback
import shutil
from pathlib import Path
from typing import List

# ml
import numpy as np
import pandas as pd

**Hide Warnings**

In [478]:

warnings.filterwarnings('ignore')

**Disable Autoscrolling**

In [479]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

**Set Random Seed**

In [480]:
random.seed(123)

<br><br><br><br>
# Question 1
# Initial Steps

Write Python code to create 20 different CSV files in this format:  `myCSV[Number].csv`, where each file contains 10 records. 

The schema is `(‘firstname’,’secondname’,city’)`  

Values should be randomly chosen from the lists: 
- `firstname` : `[John, Dana, Scott, Marc, Steven, Michael, Albert, Johanna]`  
- `city` : `[New York, Haifa, München, London, Palo Alto,  Tel Aviv, Kiel, Hamburg]`  
- `secondname`: any value  

In [481]:
# insert your path here:
my_path = "/Users/eyalmichaeli/Desktop/School/Master's/IDC_masters/big_data_platforms_ex2"

path = Path(my_path)

In [482]:
firstname = ['John', 'Dana', 'Scott', 'Marc', 'Steven', 'Michael', 'Albert', 'Johanna']
city = ['NewYork', 'Haifa', 'Munchen', 'London', 'PaloAlto', 'TelAviv', 'Kiel', 'Hamburg']
secondname = ['Lennon', 'McCartney', 'Starr', 'Harrison', 'Ono', 'Sutcliffe', 'Epstein', 'Preston']

csvs_path = Path(path / "csvs")
csvs_path.mkdir(parents=True, exist_ok=True)
for i in range(1, 21):
    temp_df = pd.DataFrame({"firstname": np.random.choice(firstname, 10),
                            "secondname": np.random.choice(secondname, 10),
                            "city": np.random.choice(city, 10),
                            })
    temp_df.to_csv(str(csvs_path / f"myCSV{i}.csv"), index=False)

print("Created 20 CSV files")

Created 20 CSV files


Use python to Create `mapreducetemp` and `mapreducefinal` folders

In [483]:
mapreducetemp_folder = Path(path / "mapreducetemp")
mapreducetemp_folder.mkdir(parents=True, exist_ok=True)

mapreducefinal_folder = Path(path / "mapreducefinal")
mapreducefinal_folder.mkdir(parents=True, exist_ok=True)

print("Created folders")

Created folders


<br><br><br>
# Question 2
## MapReduceEngine

Write Python code to create an SQLite database with the following table

`TableName: temp_results`   
`schema: (key:TEXT,value:TEXT)`

In [484]:
# Create the database "temp_results.db", then close it.
conn = None
cursor = None
try:
    conn = sqlite3.connect(str(path / "temp_results.db"))
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS temp_results (key, value);")

except Exception:
    traceback.print_exc()

finally:
    cursor.close()
    if conn:
        conn.close()

1. **Create a Python class** `MapReduceEngine` with method `def execute(input_data, map_function, reduce_function)`, such that:
    - `input_data`: is an array of elements
    - `map_function`: is a pointer to the Python function that returns a list where each entry of the form (key,value) 
    - `reduce_function`: is pointer to the Python function that returns a list where each entry of the form (key,value)

<br><br>

**Implement** the following functionality in the `execute(...)` function:

<br>

1. For each key  from the  input_data, start a new Python thread that executes map_function(key) 
<br><br>
2. Each thread will store results of the map_function into mapreducetemp/part-tmp-X.csv where X is a unique number per each thread. 
<br><br>
3. Keep the list of all threads and check whether they are completed.
<br><br>
4. Once all threads completed, load content of all CSV files into the temp_results table in SQLite.

    Remark: Easiest way to loop over all CSV files and load them into Pandas first, then load into SQLite
    `data = pd.read_csv(path to csv)`
    `data.to_sql(‘temp_results’,sql_conn, if_exists=’append’,index=False)`
<br><br>

5. **Write SQL statement** that generates a sorted list by key of the form `(key, value)` where value is concatenation of ALL values in the value column that match specific key. For example, if table has records
<table>
    <tbody>
            <tr>
                <td style="text-align:center">John</td>
                <td style="text-align:center">myCSV1.csv</td>
            </tr>
            <tr>
                <td style="text-align:center">Dana</td>
                <td style="text-align:center">myCSV5.csv</td>
            </tr>
            <tr>
                <td style="text-align:center">John</td>
                <td style="text-align:center">myCSV7.csv</td>
            </tr>
    </tbody>
</table>

    Then SQL statement will return `(‘John’,’myCSV1.csv, myCSV7.csv’)`  
    Remark: use GROUP_CONCAT and also GROUP BY ORDER BY
<br><br><br>
6. **Start a new thread** for each value from the generated list in the previous step, to execute `reduce_function(key,value)` 
<br>    
7. Each thread will store results of reduce_function into `mapreducefinal/part-X-final.csv` file  
<br>
8. Keep list of all threads and check whether they are completed  
<br>
9. Once all threads completed, print on the screen `MapReduce Completed` otherwise print `MapReduce Failed` 



In [485]:

class MapReduceEngine:
    """
    a class that implements the Map Reduce Engine. Gets an Sqlite connection in its __init__.
    Also, calls the functions: inverted_map(), inverted_reduce() in its execute method,
    which performs the Map Reduce Engine.
    """

    def __init__(self, conn):
        self.conn = conn

    def execute(self, input_data: List[str], map_function, reduce_function, params: dict, print_file_name=False):
        thread_list = list()
        csvs_paths = list()
        exec_1 = concurrent.futures.ThreadPoolExecutor()
        for csv_key in input_data:
            t = exec_1.submit(map_function, csv_key, params['column_index'])
            threads_returns = t.result()
            csv_index = input_data.index(csv_key)  # an index of the relative csv in the input_array
            csv_path = f'{mapreducetemp_folder}/part-tmp-{csv_index}.csv'
            csvs_paths.append(csv_path)
            pd.DataFrame(threads_returns).to_csv(csv_path,
                                                 header=['key', 'value'],
                                                 index=False)
            thread_list.append(t)

        # wait until the threads are completed
        exec_1.shutdown(wait=True)

        # Once all threads completed, load content of all CSV files into the temp_results table in Sqlite
        for path_to_csv in csvs_paths:
            data = pd.read_csv(path_to_csv)
            data.to_sql(name='temp_results', con=self.conn, if_exists='append', index=False)


        results_df = pd.read_sql_query("SELECT key, GROUP_CONCAT(value) as value "
                                       "FROM temp_results "
                                       "GROUP BY key "
                                       "ORDER BY key",
                                       conn)

        exec_2 = concurrent.futures.ThreadPoolExecutor()
        for res_i in range(len(results_df)):
            try:
                key = results_df["key"].iloc[res_i]
                value = results_df["value"].iloc[res_i]
                t = exec_2.submit(reduce_function, key, value, print_file_name)
                t_results = t.result() # t_results is one list, in which the 1st index is the key and the 2nd is a concat of all of the files it appears in.
                csv_path = f'{mapreducefinal_folder}/part-{res_i}-final.csv'
                csvs_paths.append(csv_path)
                pd.DataFrame({'key': t_results[0], 'value': t_results[1]}, index=[0]).to_csv(csv_path,
                                                     index=False)
                thread_list.append(t)

            except Exception:
                print(f"Mapreduce failed for result index: {res_i} with key: {key}, value: {value}")
                traceback.print_exc()
                # close connection to db
                if conn:
                    conn.close()

        # wait until the threads are completed
        exec_2.shutdown(wait=True)

        # close connection to db
        if conn:
            conn.close()

        return 'MapReduce Completed'


<br><br><br><br>

# Question 3
## Implement the MapReduce Inverted index of the JSON documents

Implement a function `inverted_map(document_name)` which reads the CSV document from the local disc and return a list that contains entries of the form (key_value, document name).

For example, if myCSV4.csv document has values like:  
`{‘firstname’:’John’,‘secondname’:’Rambo’,‘city’:’Palo Alto’}`

Then `inverted_map(‘myCSV4.csv’)` function will return a list:  
`[(‘firstname_John’,’ myCSV4.csv’),(‘secondname_Rambo’,’ myCSV4.csv’), (‘city_Palo Alto’,’ myCSV4.csv’)]`

In [486]:
def inverted_map(document_name: str, column_index: int) -> List[tuple]:
    """
    reads the CSV document from the local disc and return a list that contains entries of the form (key_value, document name) for the specific column_index provided.
    :param document_name: csv file name.
    :param column_index: column index in the csv file (Note: starting from 1)
    :return: List[tuple] where each tuple contains 2 strings
    """
    csv_path = str(path / 'csvs'/ document_name)
    df = pd.read_csv(csv_path)
    col_series = df[df.columns[column_index-1]]
    csv_path_list = [csv_path] * len(df)
    return list(zip(col_series.values, csv_path_list))

Write a reduce function `inverted_reduce(value, documents)`, where the field “documents” contains a list of all CSV documents per given value.
This list might have duplicates.
Reduce function will return new list without duplicates.

For example,
calling the function `inverted_reduce(‘firstname_Albert’,’myCSV2.csv, myCSV5.csv,myCSV2.csv’)`
will return a list `[‘firstname_Albert’,’myCSV2.csv, myCSV5.csv,myCSV2.csv’]`

In [487]:
def inverted_reduce(key: str, documents: str, print_file_name: bool) -> List[str]:
    """
    reduce function
    :param key: key value (for example: if the column is 'first_name' it could be 'Albert'.
    :param documents: a string (list) of all CSV documents per given key.
    :param print_file_name: assign print_file_name True if you want the reduce function to print the file names (it's more readable than CSV). and for debugging purposes
    :return: List: [key, documents_formatted_unique]
    """
    documents_formatted_unique = ', '.join(set(documents.replace(', ', ',').split(',')))[: -2]
    # the [: -2] is to remove the last ', '

    if print_file_name:
        list_of_file_names = [Path(csv_path).name for csv_path in documents_formatted_unique.split(', ')]
        total_files = len(list_of_file_names)
        file_names = '\n'.join(sorted(list_of_file_names))
        print(f"This key: '{key}' has appeared in {total_files} files.\n"
              f"it's in the following files: \n{file_names}\n\n")

    return [key, documents_formatted_unique]

<br><br><br><br>
# Question 4
## Testing Your MapReduce

**Create Python list** `input_data` : `[‘myCSV1.csv’,.. ,‘myCSV20.csv’]`

In [488]:
input_data = [f'myCSV{i}.csv' for i in range(1,21)]

**Submit MapReduce as follows:**

In [489]:
conn = sqlite3.connect(str(path / "temp_results.db"))
mapreduce = MapReduceEngine(conn=conn)
status = mapreduce.execute(input_data,
                           inverted_map,
                           inverted_reduce,
                           params={'column_index': 1},
                           print_file_name=True) # assign print_file_name True if you want the reduce function to print the file names (it's more readable than CSV), and for debugging purposes
print(status)

This key: 'Albert' has appeared in 15 files.
it's in the following files: 
myCSV10.csv
myCSV11.csv
myCSV12.csv
myCSV13.csv
myCSV14.csv
myCSV15.csv
myCSV16.csv
myCSV18.csv
myCSV2.csv
myCSV20.csv
myCSV3.csv
myCSV4.csv
myCSV7.csv
myCSV8.csv
myCSV9.c


This key: 'Dana' has appeared in 14 files.
it's in the following files: 
myCSV1.csv
myCSV11.csv
myCSV14.csv
myCSV15.csv
myCSV17.csv
myCSV18.csv
myCSV19.csv
myCSV2.csv
myCSV3.csv
myCSV4.csv
myCSV6.csv
myCSV7.csv
myCSV8.csv
myCSV9.c


This key: 'Johanna' has appeared in 18 files.
it's in the following files: 
myCSV1.csv
myCSV10.csv
myCSV11.csv
myCSV12.csv
myCSV13.csv
myCSV14.csv
myCSV15.csv
myCSV16.csv
myCSV17.csv
myCSV18.csv
myCSV2.csv
myCSV20.csv
myCSV3.csv
myCSV4.csv
myCSV5.csv
myCSV6.csv
myCSV8.csv
myCSV9.c


This key: 'John' has appeared in 13 files.
it's in the following files: 
myCSV1.csv
myCSV10.csv
myCSV12.csv
myCSV13.csv
myCSV14.csv
myCSV16.csv
myCSV17.csv
myCSV19.csv
myCSV20.csv
myCSV4.csv
myCSV5.csv
myCSV6.csv
myCSV9.c


This key: 

Make sure that `MapReduce Completed` should be printed and `mapreducefinal` folder should contain the result files.

**Use python to delete all temporary data from mapreducetemp folder and delete SQLite database:**

In [474]:
# delete all temp data from mapreducetemp
try:
    shutil.rmtree(str(mapreducetemp_folder))
except Exception as e:
    print(f'Error: {str(mapreducetemp_folder)}, {e.strerror}')

# delete the SQLite database
try:
    os.remove(str(path / 'temp_results.db'))
except Exception as e:
    print(f'Error: {str(path / "temp_results.db")}, {e.strerror}')


<br><br><br><br>

# Question 5
# Final Thoughts

The phase where `MapReduceEngine` reads all temporary files generated by maps and sort them to provide each reducer a specific key is called the **shuffle step**.

Please explain **clearly** what would be the main problem of MapReduce when processing Big Data, if there is no shuffle step at all, meaning reducers will directly read responses from the mappers.

Basically, the shuffling phase is where the data is being grouped by the key and summed accordingly. It's the heart of the efficiency of Map Reduce Engine! <br> The input the shufflers get is the broken down version of the data (the output of the mappers), and the output is the data aggregated, and sorted by key so that the reduce step can iterate over the keys.
If there weren't a shuffling step, the reduce function wouldn't get an aggregated count of the appearances of the key in all the files, and then the reduce function would get each key with its file path, and return the same thing. There would be no 'reduce' happening in this scenario.

In terms of volume, the shufflers significantly reduce the amount of data that is being moved forward. In the word counting example, shufflers reduce the data from the amount of words in the origin text to the amount of distinct words.
<br> If it wasn't for the shuffling phase, the reducers would have to deal with a lot more data as input

<br><br><br><br>
Good Luck :)